In [17]:
from PyPDF2 import PdfReader

reader = PdfReader(r"C:\Users\\05457936J\Desktop\CORAL\opositIA\data\\raw\\boletines\Boletin_Convocatorias_Empleo_28Mayo2025.pdf")
result=[]
for page in reader.pages:
    text = page.extract_text()
    result = result+[text]
result = ''.join(result)

In [18]:
import re

In [19]:
texto = result.split('PROCESOS  CONVOCADOS  EN LA ÚLTIMA  SEMANA')[2]
pattern = r"(.*?Referencia:\s+\d+)"
bloques = re.findall(pattern, texto, re.DOTALL)
diccionario = {}
for i, bloque in enumerate(bloques, 1):
    diccionario[i] = {'texto_original': bloque}

In [20]:
import re

def extraer_referencia(b):
    """Extrae el número de referencia."""
    m = re.search(r"Referencia:\s+(\d+)", b)
    return m.group(1) if m else None

def extraer_via(b):
    """Extrae el tipo de vía entre 'Vía:' y 'Plazas:'."""
    m = re.search(r"Vía:\s*([^:]+):", b)
    return m.group(1).strip().split('\n')[0].strip() if m else None

def extraer_plazas(b):
    """Extrae las plazas convocadas y libres."""
    m = re.search(r"Convocadas:\s+(\d+),\s+Libre:\s+(\d+)", b)
    return (m.group(1), m.group(2)) if m else (None, None)

def extraer_fecha_apertura(b):
    """Extrae la fecha de apertura (formato largo)."""
    m = re.search(r"de\s+(\d{1,2}\s+de\s+\w+\s+de\s+\d{4})", b)
    return m.group(1) if m else None

def extraer_fecha_fin(b):
    """Extrae la fecha de fin de solicitudes (formato corto)."""
    m = re.search(r"Fin del plazo de presentación de solicitudes:\s+(\d{2}/\d{2}/\d{4})", b)
    return m.group(1) if m else None

def extraer_titulo(b):
    """Extrae todo el bloque en mayúsculas antes de 'Vía:'."""
    m = re.search(r"([A-ZÁÉÍÓÚÜÑ0-9\s/().,\-]+?)\s*Vía:", b, re.DOTALL)
    return m.group(1).strip().split('\n')[-1] if m else None

{1: {'texto_original': '  \nPROCESOS DE ÁMBITO INTERNACIONAL  \nPERSONAL LABORAL  \nDOCTORADO, LICENCIATURA, INGENIERÍA, ARQUITECTURA O EQUIVALENTE  \nMINISTERIO DE DEFENSA  \nAGREGADURIA DE DEFENSA EN LA EMBAJADA DE ESPAÑA EN ARGEL (REPÚBLICA ARGE-\nLINA DEMOCRÁTICA Y POPULAR)  \nTRADUCTO R \nVía: CONTRATACIÓN FIJ A \nTitulación:  Oficial Traducto r \nPlazas:  Convocadas: 1, Libre: 1,  para  INTERNACIONA L \nCENTRO DE INFORMACIÓN ADMINISTRATIVA  de 27 de mayo de 2025 . \nFin del plazo de presentación de solicitudes:  10/06/202 5 \nObservaciones al plazo:  Las solicitudes se presentarán en el plazo de 10 días \nhábiles contados desde el día siguiente al de la publicació n \nReferencia: 215093'},
 2: {'texto_original': '  \nBACHILLERATO, BACHILLERATO UNIFICADO POLIVALENTE O FORMACIÓN \nPROFESIONAL DE TÉCNICO SUPERIOR O TÉCNICO ESPECIALISTA O EQUIVA-\nLENTE  \nMINISTERIO DE TRABAJO Y ECONOMÍA SOCIAL  \nCONSEJERÍA DE TRABAJO, MIGRACIONES Y SEGURIDAD SOCIAL EN CHINA, CON RESIDENCIA \nEN PE

In [7]:
for key, value in diccionario.items():
    texto= value['texto_original']
    diccionario[key]= value | {
        "título": extraer_titulo(texto),
        "vía": extraer_via(texto),
        "plazas_convocadas": extraer_plazas(texto)[0],
        "plazas_libres": extraer_plazas(texto)[1],
        "fecha_apertura": extraer_fecha_apertura(texto),
        "fecha_fin": extraer_fecha_fin(texto),
        "referencia": extraer_referencia(texto),
    }

In [8]:
from openai import OpenAI
API_KEY =None
client = OpenAI(api_key=API_KEY)


def extraer_inteligente(texto):
    system_prompt= "Necesito que extraigas la información de estos textos." \
            "Los campos obligatorios que hay que extraer son:" \
            "* Título: aquí tiene que venir el título de la oposición. Por ejemplo: MÉDICO, INGENIERO NAVAL, ENFERMEROS" \
            "* Organismo que convoca: por ejemplo: AYUNTAMIENTO DE MADRID" \
            "* Titulación requerida" \
            "* Vía" \
            "* Plazas convocadas" \
            "* Plazas libres" \
            "* Otras Plazas" \
            "* Fecha de publicación oficial" \
            "* Fecha de Cierre de Convocatoria" \
            "* Referencia" \
            "Si alguno de estos campos no aparece en el texto, devuelve NULL para ese campo"

    response = client.chat.completions.create(model="gpt-4.1-nano",
                                            messages=[{"role": "system", "content": system_prompt},
                                                      {"role": "user", "content": texto},
                                                      ])
    return response.choices[0].message


{1: {'texto_original': '  \nPROCESOS DE ÁMBITO INTERNACIONAL  \nPERSONAL LABORAL  \nDOCTORADO, LICENCIATURA, INGENIERÍA, ARQUITECTURA O EQUIVALENTE  \nMINISTERIO DE DEFENSA  \nAGREGADURIA DE DEFENSA EN LA EMBAJADA DE ESPAÑA EN ARGEL (REPÚBLICA ARGE-\nLINA DEMOCRÁTICA Y POPULAR)  \nTRADUCTO R \nVía: CONTRATACIÓN FIJ A \nTitulación:  Oficial Traducto r \nPlazas:  Convocadas: 1, Libre: 1,  para  INTERNACIONA L \nCENTRO DE INFORMACIÓN ADMINISTRATIVA  de 27 de mayo de 2025 . \nFin del plazo de presentación de solicitudes:  10/06/202 5 \nObservaciones al plazo:  Las solicitudes se presentarán en el plazo de 10 días \nhábiles contados desde el día siguiente al de la publicació n \nReferencia: 215093'},
 2: {'texto_original': '  \nBACHILLERATO, BACHILLERATO UNIFICADO POLIVALENTE O FORMACIÓN \nPROFESIONAL DE TÉCNICO SUPERIOR O TÉCNICO ESPECIALISTA O EQUIVA-\nLENTE  \nMINISTERIO DE TRABAJO Y ECONOMÍA SOCIAL  \nCONSEJERÍA DE TRABAJO, MIGRACIONES Y SEGURIDAD SOCIAL EN CHINA, CON RESIDENCIA \nEN PE

In [36]:
import pandas as pd

In [37]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [38]:
df = pd.DataFrame(diccionario).T

In [72]:
df

In [125]:
df["resultado_gpt"] = df["texto_original"].apply(lambda x: extraer_inteligente(x))

In [153]:
df.to_csv(r"C:\Users\\05457936J\Desktop\CORAL\opositIA\data\\processed\\boletines\Boletin_Convocatorias_Empleo_28Mayo2025.csv", header=True, sep='#')

In [163]:
print('  \nUNIVERSITAT DE VALÈNCIA \n(ESTUDI GENERAL)  \nINVESTIGADOR NO DOCTOR  \nVía: CONTRATACIÓN TEMPORAL  \nPlazas: Convocadas: 1, Libre: 1, \npara la provincia de VALENCIA   \n \nConvocatorias de Empleo Público  \n \n147 DIARI OFICIAL DE LA COMUNITAT \nVALENCIANA de 22 de mayo de \n2025.  \nFin del plazo de presentación de \nsolicitudes: 05/06/2025  \nObservaciones al plazo: El plazo de \npresentación de las solicitudes será \nde 10 días hábiles a partir del si-\nguiente hábil al de la publicación de \nesta resolución en el Diari Oficial de \nla Generalitat Valenciana (DOGV)  \nObservaciones: Proyecto: «Fase 2 \nclassification algorithm robustness, \nin situ data collection protocols, \nlandsat 8 integration. Subcontract \nword cereal nº 204303073. Esa con-\ntract nº 4000130569/20/I -NB. CPI -\n25-192», a cargo de la línea de in-\nvestigación: «Generación y anál isis \nde calendarios de cultivos».  \nReferencia:  214940')

721

In [2]:
file = 'C:\\Users\\05457936J\\Desktop\\CORAL\\opositIA\\data\\processed\\boletines\\Boletin_Convocatorias_Empleo_04Junio2025.txt'

In [3]:
with open(file, 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
text

' \n \n \n \nCONVOCATORIAS  \nadministracion.gob.es  \nTeléfono: 060  \n \n \n \nSemana del 0 4.06.202 5 al 10.06.202 5 \n \n \n  \n \n \n \n \n \n \n \n \n \n \nCont acto : https://administracion.gob.es  \nN.I.P.O: 137 -22-010-5 Ejemplar gratuito   \n \nCentro de Información Administrativa  \n \n ÍNDICE  \nGENERALIDADES  ................................ ................................ ................................ .................  V \nQué se Incluye en esta Publicación  ................................ ................................ ..................  V \nDisposiciones Reglamentarias  ................................ ................................ .........................  V \nHORARIOS  DE ATENCIÓN  ................................ ................................ ................................ .. VII \nCentro de Información Administrativa  ................................ ................................ ...........  VII \nRegistro  ................................ ....

In [6]:
import re

In [7]:
texto = text.split('PROCESOS  CONVOCADOS  EN LA ÚLTIMA  SEMANA')[2]
pattern = r"(.*?Referencia:\s+\d+)"
bloques = re.findall(pattern, texto, re.DOTALL)
diccionario = {}
for i, bloque in enumerate(bloques, 1):
    diccionario[i] = {'texto_original': bloque}

In [8]:
diccionario

{}